<a id="top"></a> <br>
## contents
1. [Data ETL](#1)
1. [Create Vertical Federation](#2)
1. [Create Horizontal Federation](#3)

<a id="1"></a> <br>
## 1- Data ETL

###### [Go to top](#top)

This notebook was mainly copied from [this notebook](https://www.kaggle.com/code/chauhuynh/my-first-kernel-3-699). All credits belongs to the original author.

In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

import random
seed = 1414
random.seed(seed)
np.random.seed(seed)

In [2]:
! ls /kaggle/input/elofederatedlearningdataetltraintest

horizontalsplit-0-0-lower_table.table.csv


In [3]:
df_train = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')
#df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')

In [4]:
test_df = pd.read_csv('../input/elofederatedlearningdataetltraintest/horizontalsplit-0-0-lower_table.table.csv')
test_df['test'] = 1
test_df = test_df[['card_id', 'test']]

In [5]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [6]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [7]:
extra_train = df_train[df_train['outliers'] == 1]

In [8]:
df_train = df_train[df_train['outliers'] != 1]

In [9]:
extra_train.shape

(2207, 7)

In [10]:
df_train.shape

(199710, 7)

In [11]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [12]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [13]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
#df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

14

In [14]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new_hist',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
#df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

147

In [15]:
del df_hist_trans;gc.collect()
del df_new_merchant_trans;gc.collect()
df_train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_installments_sum,hist_installments_max,hist_installments_min,hist_installments_mean,hist_installments_var,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_diff_mean,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,new_hist_month_nunique,new_hist_hour_nunique,new_hist_weekofyear_nunique,new_hist_dayofweek_nunique,new_hist_year_nunique,new_hist_subsector_id_nunique,new_hist_merchant_id_nunique,new_hist_merchant_category_id_nunique,new_hist_purchase_amount_sum,new_hist_purchase_amount_max,new_hist_purchase_amount_min,new_hist_purchase_amount_mean,new_hist_purchase_amount_var,new_hist_installments_sum,new_hist_installments_max,new_hist_installments_min,new_hist_installments_mean,new_hist_installments_var,new_hist_purchase_date_max,new_hist_purchase_date_min,new_hist_month_lag_max,new_hist_month_lag_min,new_hist_month_lag_mean,new_hist_month_lag_var,new_hist_month_diff_mean,new_hist_weekend_sum,new_hist_weekend_mean,new_hist_category_1_sum,new_hist_category_1_mean,new_hist_card_id_size,new_hist_category_2_mean_mean,new_hist_category_3_mean_mean,new_hist_purchase_date_diff,new_hist_purchase_date_average,new_hist_purchase_date_uptonow
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,0,9,23,35,7,2,21,94,41,-165.968739,2.258395,-0.739395,-0.638341,0.045003,4,1,0,0.015385,0.015206,2018-02-25 09:31:15,2017-06-27 14:18:08,0,-8,-3.911538,5.748901,51.950000,247,0.950000,90,0.346154,0,0.000000,260,0.072502,0.346719,242,0.930769,1562,2.0,8.0,7.0,7.0,1.0,10.0,23.0,14.0,-13.244202,-0.296112,-0.724368,-0.575835,0.018445,0.0,0.0,0.0,0.000000,0.000000,2018-04-29 11:23:05,2018-03-05 14:04:36,2.0,1.0,1.478261,0.260870,51.782609,6.0,0.260870,0.0,0.000000,23.0,-0.550160,-0.592993,54.0,2.347826,1499.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0,12,24,50,7,2,24,142,57,-210.006336,4.630299,-0.742400,-0.600018,0.148200,543,10,-1,1.551429,2.282448,2018-01-31 22:31:09,2017-01-06 16:29:42,0,-12,-5.031429,14.477519,53.000000,339,0.968571,132,0.377143,31,0.088571,350,0.074568,-0.295163,390,1.114286,1586,2.0,5.0,4.0,4.0,1.0,4.0,6.0,5.0,-4.355735,-0.701858,-0.739410,-0.725956,0.000205,6.0,1.0,1.0,1.000000,0.000000,2018-03-30 06:48:26,2018-02-01 17:07:54,2.0,1.0,1.500000,0.300000,52.833333,0.0,0.000000,0.0,0.000000,6.0,-0.550160,-0.606486,56.0,9.333333,1529.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0,10,14,22,7,2,7,13,8,-29.167391,-0.145847,-0.730138,-0.678311,0.007635,0,0,0,0.000000,0.000000,2018-02-27 19:08:25,2017-01-11 08:21:22,0,-13,-8.604651,14.768549,52.000000,41,0.953488,11,0.255814,0,0.000000,43,-0.087803,0.358458,412,9.581395,1559,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.700326,-0.700326,-0.700326,-0.700326,NaN,0.0,0.0,0.0,0.000000,NaN,2018-04-28 17:43:11,2018-04-28 17:43:11,2.0,2.0,2.000000,NaN,51.000000,1.0,1.000000,0.0,0.000000,1.0,-0.549015,-0.592993,0.0,0.000000,1499.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,0,6,16,20,7,2,13,50,25,-49.491364,1.445596,-0.740897,-0.642745,0.068447,84,3,-1,1.090909,0.346890,2018-02-28 11:44:40,2017-09-26 16:22:21,0,-5,-2.831169,3.247437,51.935065,77,1.000000,11,0.142857,12,0.155844,77,-0.086166,-0.338321,154,2.000000,1559,2.0,5.0,5.0,4.0,1.0,5.0,7.0,6.0,-4.654372,-0.566740,-0.734135,-0.664910,0.004340,5.0,1.0,-1.0,0.714286,0.571429,2018-04-18 11:00:11,2018-03-07 11:55:06,2.0,1.0,1.714286,0.238095,52.000000,3.0,0.428571,1.0,0.142857,7.0,-0.5

In [16]:
df_train.shape

(199710, 79)

In [17]:
for df in [df_train]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

In [18]:
df_train_columns = [c for c in df_train.columns if c not in ['first_active_month', 'outliers']]
df_train = df_train[df_train_columns]

In [19]:
df_train.shape

(199710, 85)

In [20]:
df_train = df_train.merge(test_df,on='card_id',how='left')

In [21]:
df_train['test'].value_counts()

1.0    19971
Name: test, dtype: int64

In [22]:
df_train.shape

(199710, 86)

In [23]:
df_train.head()

,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_installments_sum,hist_installments_max,hist_installments_min,hist_installments_mean,hist_installments_var,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_diff_mean,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,...,new_hist_year_nunique,new_hist_subsector_id_nunique,new_hist_merchant_id_nunique,new_hist_merchant_category_id_nunique,new_hist_purchase_amount_sum,new_hist_purchase_amount_max,new_hist_purchase_amount_min,new_hist_purchase_amount_mean,new_hist_purchase_amount_var,new_hist_installments_sum,new_hist_installments_max,new_hist_installments_min,new_hist_installments_mean,new_hist_installments_var,new_hist_purchase_date_max,new_hist_purchase_date_min,new_hist_month_lag_max,new_hist_month_lag_min,new_hist_month_lag_mean,new_hist_month_lag_var,new_hist_month_diff_mean,new_hist_weekend_sum,new_hist_weekend_mean,new_hist_category_1_sum,new_hist_category_1_mean,new_hist_card_id_size,new_hist_category_2_mean_mean,new_hist_category_3_mean_mean,new_hist_purchase_date_diff,new_hist_purchase_date_average,new_hist_purchase_date_uptonow,dayofweek,weekofyear,month,elapsed_time,hist_first_buy,new_hist_first_buy,card_id_total,purchase_amount_total,test
0,C_ID_92a2005557,5,2,1,-0.820283,9,23,35,7,2,21,94,41,-165.968739,2.258395,-0.739395,-0.638341,0.045003,4,1,0,0.015385,0.015206,1.519551e+09,1.498573e+09,0,-8,-3.911538,5.748901,51.950000,247,0.950000,90,0.346154,0,0.000000,260,0.072502,0.346719,242,...,1.0,10.0,23.0,14.0,-13.244202,-0.296112,-0.724368,-0.575835,0.018445,0.0,0.0,0.0,0.000000,0.000000,1.525001e+09,1.520259e+09,2.0,1.0,1.478261,0.260870,51.782609,6.0,0.260870,0.0,0.000000,23.0,-0.550160,-0.592993,54.0,2.347826,1499.0,3,22,6,1831,26,277.0,283.0,-179.212942,NaN
1,C_ID_3d0044924f,4,1,0,0.392913,12,24,50,7,2,24,142,57,-210.006336,4.630299,-0.742400,-0.600018,0.148200,543,10,-1,1.551429,2.282448,1.517438e+09,1.483720e+09,0,-12,-5.031429,14.477519,53.000000,339,0.968571,132,0.377143,31,0.088571,350,0.074568,-0.295163,390,...,1.0,4.0,6.0,5.0,-4.355735,-0.701858,-0.739410,-0.725956,0.000205,6.0,1.0,1.0,1.000000,0.000000,1.522393e+09,1.517505e+09,2.0,1.0,1.500000,0.300000,52.833333,0.0,0.000000,0.0,0.000000,6.0,-0.550160,-0.606486,56.0,9.333333,1529.0,6,52,1,1982,5,396.0,356.0,-214.362071,NaN
2,C_ID_d639edf6cd,2,2,0,0.688056,10,14,22,7,2,7,13,8,-29.167391,-0.145847,-0.730138,-0.678311,0.007635,0,0,0,0.000000,0.000000,1.519759e+09,1.484123e+09,0,-13,-8.604651,14.768549,52.000000,41,0.953488,11,0.255814,0,0.000000,43,-0.087803,0.358458,412,...,1.0,1.0,1.0,1.0,-0.700326,-0.700326,-0.700326,-0.700326,NaN,0.0,0.0,0.0,0.000000,NaN,1.524937e+09,1.524937e+09,2.0,2.0,2.000000,NaN,51.000000,1.0,1.000000,0.0,0.000000,1.0,-0.549015,-0.592993,0.0,0.000000,1499.0,0,31,8,2135,163,635.0,44.0,-29.867717,NaN
3,C_ID_186d6a6901,4,3,0,0.142495,6,16,20,7,2,13,50,25,-49.491364,1.445596,-0.740897,-0.642745,0.068447,84,3,-1,1.090909,0.346890,1.519818e+09,1.506443e+09,0,-5,-2.831169,3.247437,51.935065,77,1.000000,11,0.142857,12,0.155844,77,-0.086166,-0.338321,154,...,1.0,5.0,7.0,6.0,-4.654372,-0.566740,-0.734135,-0.664910,0.004340,5.0,1.0,-1.0,0.714286,0.571429,1.524049e+09,1.520424e+09,2.0,1.0,1.714286,0.238095,52.000000,3.0,0.428571,1.0,0.142857,7.0,-0.556518,-0.604559,41.0,5.857143,1510.0,4,35,9,1739,25,187.0,84.0,-54.145736,NaN
4,C_ID_cdbd2c0db2,1,3,0,-0.159749,4,22,17,7,2,17,66,26,-48.687656,7.193041,-0.746156,-0.366073,1.828160,182,12,1,1

In [24]:
df_train.to_csv('elo-ETL-data.csv')

In [25]:
df_train[df_train['test']==1].drop(['test'],axis=1).to_csv('elo-ETL-data-test.csv')
df_train[df_train['test']!=1].drop(['test'],axis=1).to_csv('elo-ETL-data-train.csv')

<a id="2"></a> <br>
## 2- Create Vertical Federation

###### [Go to top](#top)

In [26]:
df_train_columns_without_id = [c for c in df_train.columns if c not in ['card_id','target','test']]
round(len(df_train_columns_without_id)*0.6)

50

In [27]:
df_train_columns_60 = random.sample(df_train_columns_without_id, round(len(df_train_columns_without_id)*0.6))
df_train_columns_40 = [c for c in df_train.columns if c not in df_train_columns_60]
df_train_columns_60 = df_train_columns_60 + ['card_id', 'test']
print(df_train_columns_60)

['hist_installments_max', 'hist_month_lag_var', 'hist_purchase_amount_sum', 'new_hist_month_lag_max', 'hist_merchant_category_id_nunique', 'hist_authorized_flag_mean', 'new_hist_installments_var', 'new_hist_merchant_id_nunique', 'new_hist_year_nunique', 'new_hist_month_lag_min', 'hist_subsector_id_nunique', 'new_hist_merchant_category_id_nunique', 'feature_1', 'hist_purchase_amount_var', 'hist_category_1_mean', 'hist_hour_nunique', 'feature_2', 'new_hist_category_3_mean_mean', 'hist_purchase_date_average', 'new_hist_weekend_sum', 'hist_installments_min', 'hist_weekend_sum', 'hist_first_buy', 'new_hist_category_2_mean_mean', 'new_hist_purchase_date_average', 'hist_purchase_amount_max', 'new_hist_weekofyear_nunique', 'new_hist_purchase_date_diff', 'hist_purchase_date_diff', 'hist_month_lag_min', 'hist_purchase_date_uptonow', 'weekofyear', 'new_hist_card_id_size', 'hist_weekofyear_nunique', 'new_hist_purchase_amount_max', 'hist_card_id_size', 'hist_merchant_id_nunique', 'card_id_total', '

In [28]:
print(df_train_columns_40)

['card_id', 'target', 'hist_dayofweek_nunique', 'hist_year_nunique', 'hist_purchase_amount_min', 'hist_purchase_amount_mean', 'hist_installments_sum', 'hist_installments_mean', 'hist_installments_var', 'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_month_lag_mean', 'hist_month_diff_mean', 'hist_authorized_flag_sum', 'hist_weekend_mean', 'hist_category_1_sum', 'hist_category_2_mean_mean', 'hist_category_3_mean_mean', 'new_hist_month_nunique', 'new_hist_dayofweek_nunique', 'new_hist_subsector_id_nunique', 'new_hist_purchase_amount_min', 'new_hist_installments_sum', 'new_hist_installments_min', 'new_hist_installments_mean', 'new_hist_purchase_date_max', 'new_hist_purchase_date_min', 'new_hist_month_lag_mean', 'new_hist_month_lag_var', 'new_hist_month_diff_mean', 'new_hist_category_1_mean', 'dayofweek', 'month', 'new_hist_first_buy', 'purchase_amount_total', 'test']


In [29]:
df_train[df_train_columns_60].shape

(199710, 52)

In [30]:
extra_train_columns = [c for c in extra_train.columns if c not in ['first_active_month', 'outliers', 'target','test']]

In [31]:
extra_train[extra_train_columns].shape

(2207, 4)

In [32]:
extra_train[extra_train_columns].head()

,card_id,feature_1,feature_2,feature_3
64,C_ID_8186f3fcc1,3,1,1
92,C_ID_b9379a30ea,3,3,1
125,C_ID_e9120f535c,3,3,1
151,C_ID_65715cb80d,4,2,0
225,C_ID_ae77d244b6,2,1,0


In [33]:
fed_60_v = pd.concat([df_train[df_train_columns_60],extra_train[extra_train_columns]])
fed_60_v.shape

(201917, 52)

In [34]:
fed_60_v_test = fed_60_v[fed_60_v['test']==1]
fed_60_v_test = fed_60_v_test.drop(['test'],axis=1)
fed_60_v_test.to_csv('elo-ETL-data-60-vertical-test.csv')
for i,each in enumerate([c for c in df_train_columns_60 if c not in ['card_id','target']]):
    #print(i)
    fed_60_v_test.rename(columns={each:f'x{i}'},inplace=True)
    
#fed_60_v_test.to_csv('elo-ETL-data-60-vertical-test-x.csv')

In [35]:
fed_60_v_test.shape

(19971, 51)

In [36]:
fed_60_v_train = fed_60_v[fed_60_v['test']!=1]
fed_60_v_train = fed_60_v_train.drop(['test'],axis=1)
fed_60_v_train.to_csv('elo-ETL-data-60-vertical-train.csv')
for i,each in enumerate([c for c in df_train_columns_60 if c not in ['card_id','target']]):
    #print(i)
    fed_60_v_train.rename(columns={each:f'x{i}'},inplace=True)
    
#fed_60_v_train.to_csv('elo-ETL-data-60-vertical-train-x.csv')

In [37]:
fed_60_v_train.shape

(181946, 51)

In [38]:
fed_40_v = df_train[df_train_columns_40]
fed_40_v_test = fed_40_v[fed_40_v['test']==1]
fed_40_v_test = fed_40_v_test.drop(['test'],axis=1)
fed_40_v_test.to_csv('elo-ETL-data-40-vertical-test.csv')
for i,each in enumerate([c for c in df_train_columns_40 if c not in ['card_id','target']]):
    #print(i)
    fed_40_v_test.rename(columns={each:f'x{i}'},inplace=True)
    
#fed_40_v_test.to_csv('elo-ETL-data-40-vertical-test-x.csv')

In [39]:
fed_40_v_test.shape

(19971, 35)

In [40]:
fed_40_v_train = fed_40_v[fed_40_v['test']!=1]
fed_40_v_train = fed_40_v_train.drop(['test'],axis=1)
fed_40_v_train.to_csv('elo-ETL-data-40-vertical-train.csv')
for i,each in enumerate([c for c in df_train_columns_40 if c not in ['card_id','target']]):
    #print(i)
    fed_40_v_train.rename(columns={each:f'x{i}'},inplace=True)
    
#fed_40_v_train.to_csv('elo-ETL-data-40-vertical-train-x.csv')

In [41]:
fed_40_v_train.shape

(179739, 35)

<a id="3"></a> <br>
## 3- Create Horizontal Federation

###### [Go to top](#top)

In [42]:
df_train.shape

(199710, 86)

In [43]:
df_t = df_train[df_train['test']==1]
df_t.drop(['test'],axis=1).to_csv('elo-ETL-data-horizontal-test.csv')

In [44]:
df_t.drop(['test'],axis=1).shape

(19971, 85)

In [45]:
df_train = df_train[df_train['test']!=1].drop(['test'],axis=1)

In [46]:
fed_60_h = df_train.sample(n=round(df_train.shape[0]*0.6), random_state=seed, axis=0)
fed_60_h.to_csv('elo-ETL-data-60-horizontal-train.csv')

In [47]:
fed_60_h.shape

(107843, 85)

In [48]:
fed_40_h = pd.concat([df_train, fed_60_h, fed_60_h]).drop_duplicates(keep=False)  #df1-df2
fed_40_h.to_csv('elo-ETL-data-40-horizontal-train.csv')

In [49]:
fed_40_h.shape

(71896, 85)

In [50]:
for i,each in enumerate([c for c in fed_60_h.columns if c not in ['card_id','target']]):
    #print(i)
    fed_60_h.rename(columns={each:f'x{i}'},inplace=True)
    fed_40_h.rename(columns={each:f'x{i}'},inplace=True)

In [51]:
#fed_60_h.to_csv('elo-ETL-data-60-horizontal-train-x.csv')
#fed_40_h.to_csv('elo-ETL-data-40-horizontal-train-x.csv')